In [1]:
import cv2
import numpy as np
from PIL import Image 
import pickle
import os


In [2]:
# We Have to Run this Cell for Once Only as it will generate the Pickle File for once and all.
def createpickle():
    namelist = []
    file_name = "namelist.pkl"
    f = open(file_name,'wb')

    pickle.dump(namelist,f)
    f.close()



# For Dataset generation Using Web Camera

In [3]:
def datagenerate():    
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    with open('namelist.pkl', 'rb') as f:
        namelist= pickle.load(f)
        
    name = input("Enter Full Name for the Data Generation: ")
    namelist.append(name)
    id = namelist.index(name)
    
    a = os.getcwd()
    idpath = f'{a}\\data\\{id}'
    if not os.path.exists(idpath):
        os.makedirs(idpath)
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.2,5)
        for(x,y,w,h) in faces:
            face_cropped = img[y:y+h,x:x+w]
            return face_cropped
        
    cap = cv2.VideoCapture(0)
    # cap.set(3, 640) # set video width
    # cap.set(4, 480) # set video height
    img_id = 0
    
    while True:
        ret,my_frame = cap.read()
        
        if face_cropped(my_frame) is not None:
            img_id+=1
            
            face = cv2.resize(face_cropped(my_frame),(450,450))
            
            face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
      
            file_path=f"data/{id}/"+name+"."+str(id)+"."+str(img_id)+".jpg"
            
    
            cv2.imwrite(file_path,face)
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),2)
            cv2.imshow("Generated Face",face)
            
        if cv2.waitKey(1)==13 or int(img_id) == 100:
            break
    with open('namelist.pkl','wb') as f:
        pickle.dump(namelist,f)
        
        
    
    cap.release()
    cv2.destroyAllWindows()

# For Dataset Generation using an Image

In [4]:
def imagegenerate():    
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    with open('namelist.pkl', 'rb') as f:
        namelist= pickle.load(f)
        
    name = input("Enter Full Name for the Data Generation: ")
    namelist.append(name)
    id = namelist.index(name)
    
    a = os.getcwd()
    idpath = f'{a}\\data\\{id}'
    if not os.path.exists(idpath):
        os.makedirs(idpath)
        
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)
        for(x,y,w,h) in faces:
            face_cropped = img[y:y+h,x:x+w]
            return face_cropped
        
    #cap = cv2.VideoCapture(0)
    # cap = cv2.imread("tobey.jpg")
    
    
    img_id = 0
    
    while True:
    #     ret,my_frame = cap.read()
        my_frame=cv2.imread("rock.jpg")
        
        if face_cropped(my_frame) is not None:
            img_id+=1
            
            face = cv2.resize(face_cropped(my_frame),(450,450))
            
            face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
      
            file_path=f"data/{id}/"+name+"."+str(id)+"."+str(img_id)+".jpg"
            
    
            cv2.imwrite(file_path,face)
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),2)
            cv2.imshow("Generated Face",face)
            
        if cv2.waitKey(1)==13 or int(img_id) == 50:
            break
    with open('namelist.pkl','wb') as f:
        pickle.dump(namelist,f)
        
        
    
    
    cv2.destroyAllWindows()

# Training the Model with Image folder

In [5]:
def trainer():
    # Path for face image database
    path = 'data'

    recognizer = cv2.face.LBPHFaceRecognizer_create() 
    detector = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_default.xml")
    def imagenames(path):
    
        face_samples=[]
        faceids=[]
    

        for path,subdirnames,filenames in os.walk(path):
            for filename in filenames:

                id=int(os.path.basename(path))
                imagePaths=os.path.join(path,filename)

                pilimage = Image.open(imagePaths).convert('L')
                
                img_numpy = np.array(pilimage,'uint8')
            
                faces = detector.detectMultiScale(img_numpy)

                for (x,y,w,h) in faces:
                    face_samples.append(img_numpy[y:y+h,x:x+w])
                    faceids.append(id)

            
    
        return face_samples,faceids

    print ("\nTraining Started.\n")
    faces,faceids = imagenames(path)
    recognizer.train(faces, np.array(faceids))
    recognizer.save('trainer/trainer.yml') 
    print("{0} faces trained. Exiting Training Program".format(len(np.unique(faceids))))

# Face Recognition (Live Camera) using LPBH Algorithm 


In [6]:
def facerecognition():
    print('\nStarting Recognizer....')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer/trainer.yml')
 
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_default.xml")

    font = cv2.FONT_HERSHEY_SIMPLEX

    # Starting realtime video capture
    cam = cv2.VideoCapture(0)
    
    # Define min window size to be recognized as a face

    with open('namelist.pkl', 'rb') as f:
        names = pickle.load(f)

    while True:

        ret, img = cam.read()
        

        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale( 
            gray,
            scaleFactor = 1.2,
            minNeighbors = 5,
           )

        for(x,y,w,h) in faces:

            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

            id, predic = recognizer.predict(gray[y:y+h,x:x+w])
            confidence = int((100*(1-predic/300)))

            # Check if confidence is less them 100 ==> "0" is perfect match 
            if (confidence > 77):
                id = names[id]
               # confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
               # confidence = "  {0}%".format(round(100 - confidence))

            cv2.putText(img, str(id),(x+w,y+w),cv2.FONT_HERSHEY_DUPLEX,3,(255,0,0),6)
            cv2.putText(img, str(predic),(x,y),cv2.FONT_HERSHEY_DUPLEX,1,(255,0,0),2)

        cv2.imshow('camera',img) 

        k = cv2.waitKey(13) & 0xff 
        if k == 27:
            break

  
    print("\nExiting Recognizer.")
    cam.release()
    cv2.destroyAllWindows()

# FACE RECOGNITION IMAGE USING LBPH

In [7]:
def facerecognitionimage():
    print('\nStarting Recognizer....')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer/trainer.yml')
 
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_default.xml")

    font = cv2.FONT_HERSHEY_SIMPLEX



    with open('namelist.pkl', 'rb') as f:
        names = pickle.load(f)

    while True:

        img = cv2.imread("images.jpg")
        
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale( 
            gray,
            scaleFactor = 1.2,
            minNeighbors = 5,
#             minSize = (int(minW), int(minH)),
           )

        for(x,y,w,h) in faces:

            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

            id, predic = recognizer.predict(gray[y:y+h,x:x+w])
#            
            confidence = int((100*(1-predic/300)))
#             confidence = round(confidence,2)

            # Check if confidence is less them 100 ==> "0" is perfect match
            if (confidence > 77):
                id = names[id]
               # confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
               # confidence = "  {0}%".format(round(100 - confidence))

            cv2.putText(img, str(id),(x,y+w),cv2.FONT_HERSHEY_DUPLEX,2,(255,0,0),2)
            cv2.putText(img, str(confidence),(x,y),cv2.FONT_HERSHEY_DUPLEX,1,(255,0,0),2)

        cv2.imshow('camera',img) 

        k = cv2.waitKey(13) & 0xff 
        if k == 27:
            break

  
    print("\nExiting Recognizer.")
    
    cv2.destroyAllWindows()

In [8]:
c = 0
while(c!= 7):
    c = int(input("\n1. Create Pickle File \n2. Generate Dataset (Live Camera) \n3. Generate Dataset(Images) \n4. Train Data \n5. Face Recognition (Live Camera) \n6. Face Recognition (Image)  \n7. Exit \n"))
    if c == 1 :
        createpickle()
    if c == 2:
        datagenerate()
    if c == 3:
        imagegenerate()
    if c == 4:
        trainer()
    if c == 5:
        facerecognition()
    if c == 6:
        facerecognitionimage()
    if c == 7:
        break



1. Create Pickle File 
2. Generate Dataset (Live Camera) 
3. Generate Dataset(Images) 
4. Train Data 
5. Face Recognition (Live Camera) 
6. Face Recognition (Image)  
7. Exit 
7


In [9]:
print("Mister Maker")

Mister Maker
